In [36]:
import sys

sys.path.insert(0,'../src/')

import pandas_gbq
import pendulum
import pickle

from weather_data_utils import *

In [37]:
project_id = 'emissions-monitoring'

sql = '''
SELECT
  gems_plant_id,
  latitude,
  longitude,
  plant_capacity_mw_fuel_coal
FROM
  plant_metadata.plant_attributes
WHERE
  plant_capacity_mw_fuel_coal >= 1000
  AND plant_capacity_mw_cool_ndt / plant_capacity_mw_total >= 0.7
  AND latitude IS NOT NULL
  AND longitude IS NOT NULL
  AND country='CHINA'
order by plant_capacity_mw_fuel_coal desc
limit 10
'''

plants_df = pandas_gbq.read_gbq(sql, project_id=project_id)

/Users/joshgree/Library/Caches/pypoetry/virtualenvs/airflow-pipelines-ELMc40Ku-py3.6/lib/python3.6/site-packages/google/cloud/bigquery/table.py:1772: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  create_bqstorage_client=create_bqstorage_client,


In [38]:
plants_df

,gems_plant_id,latitude,longitude,plant_capacity_mw_fuel_coal
0,1023040,40.1892,111.3596,6720.0
1,1028551,35.3256,116.9261,4540.0
2,1010203,32.6842,116.9021,4480.0
3,1018298,35.1679,112.7162,4400.0
4,1067575,39.2193,117.9293,4000.0
5,1051110,25.1995,104.6826,3600.0
6,1018921,48.5497,119.7739,3400.0
7,1074887,34.3781,117.1763,3340.0
8,1073472,31.9890,120.6884,3260.0
9,1024370,40.6597,114.9439,3000.0


In [39]:
pendulum.yesterday()

Signature: pendulum.yesterday(tz='local')
Docstring: Create a DateTime instance for today.
File:      ~/Library/Caches/pypoetry/virtualenvs/airflow-pipelines-ELMc40Ku-py3.6/lib/python3.6/site-packages/pendulum/__init__.py
Type:      function


In [40]:
start = pendulum.datetime(2016,1, 1)
end = pendulum.yesterday(tz='UTC')
period = pendulum.period(start, end)

dates = [x.date() for x in period.range("days",amount=35)]
starts = dates[:-1]
ends = [x.subtract(days=1) for x in dates[1:]]

periods = [(str(start),str(end)) for start,end in zip(starts,ends)]

In [5]:
# raw data
raw_data = {}
for row in df.to_dict(orient="records"):
    for period in periods:
        lat,lon = row['latitude'],row['longitude']
        raw_data[((lat,lon),period)] = get_raw_weather_json(lat=lat,lon=lon,start=period[0],end=period[1])

In [19]:
# dump to pickle
with open('raw_data.p','wb') as f:
    pickle.dump(raw_data,f)

In [41]:
with open('raw_data.p','rb') as f:
    raw_data_ = pickle.load(f)

In [42]:
raw_data = raw_data_

In [43]:
dfs = []
for k in list(raw_data.keys()):
    lat = k[0][0]
    lon = k[0][1]
    df = process_raw_weather_json(raw_json=raw_data[k],lat=lat,lon=lon)
    
    df = df.assign(
        **{'lat':lat,'lon':lon}
    )

    dfs.append(df)

In [45]:
df = pd.concat(dfs)
df = pd.merge(df, plants_df, how='left',left_on=['lat','lon'],right_on=['latitude','longitude'])
df.to_csv('cleaned_weather_data.csv',index=False)

In [47]:
len(periods) * 2

104